In [3]:
!pip install mesa

     |████████████████████████████████| 691 kB 418 kB/s eta 0:00:01
  Using cached cookiecutter-1.7.3-py2.py3-none-any.whl (34 kB)
  Using cached binaryornot-0.4.4-py2.py3-none-any.whl (9.0 kB)
  Using cached poyo-0.5.0-py2.py3-none-any.whl (10 kB)
  Using cached jinja2_time-0.2.0-py2.py3-none-any.whl (6.4 kB)
     |████████████████████████████████| 64 kB 10.4 MB/s eta 0:00:01
  Using cached text_unidecode-1.3-py2.py3-none-any.whl (78 kB)


In [4]:
from mesa import Agent, Model

In [5]:
class OpinionAgent(Agent):
    """ An agent with fixed initial wealth."""
    def __init__(self, x, u, mu, unique_id, model):
        super().__init__(unique_id, model)
        self.x = x
        self.u = u
        self.mu = mu
    
    def influence(self, agent):
        overlap = min(self.x + self.u, agent.x + agent.u) - max(self.x - self.u, agent.x - agent.u)
        delta = overlap / self.u - 1
        return [self.mu * delta * (agent.x - self.x), self.mu * delta * (agent.u - self.u)] if delta > 0 else [0, 0]
    
    def step(self, agents):
        # do something with influence
        fin_influence = np.array([influence(agent) for agent in agents])
        self.x += sum(fin_influence[:, 0])
        self.x = max(self.x , -1)
        self.x = min(self.x , 1)
        self.u += sum(fin_influence[:, 1])

In [ ]:
class OpinionModel(Model):
    def __init__(self, N, width, height):
        self.num_agents = N
        # Create agents
        for i in range(self.num_agents):
            a = MoneyAgent(i, self)
            self.schedule.add(a)
            # Add the agent to a random grid cell
            x = self.random.randrange(self.grid.width)
            y = self.random.randrange(self.grid.height)
            self.grid.place_agent(a, (x, y))

        self.datacollector = DataCollector(
            model_reporters={"Gini": compute_gini},
            agent_reporters={"Wealth": "wealth"}
        )

    def step(self):
        self.datacollector.collect(self)
        self.schedule.step()
        